In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import random
import spacy
import re
import os
import json

# Set data directory
data_dir = "../data/opensubtitles_en_ko"

# Load tokenizer models
spacy_en = spacy.load("en_core_web_sm")
spacy_ko = spacy.load("ko_core_news_sm")

# Tokenizer functions
def tokenize_en(text):
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

def tokenize_ko(text):
    return [tok.text for tok in spacy_ko.tokenizer(text)]

# Load dataset
src_file = os.path.join(data_dir, "OpenSubtitles.en-ko.en")
trg_file = os.path.join(data_dir, "OpenSubtitles.en-ko.ko")

src_texts, trg_texts = [], []
with open(src_file, "r", encoding="utf-8") as f_en, open(trg_file, "r", encoding="utf-8") as f_ko:
    for en_line, ko_line in zip(f_en, f_ko):
        src_texts.append(en_line.strip())
        trg_texts.append(ko_line.strip())

# Save dataset
save_data = lambda data, path: json.dump(data, open(path, "w", encoding="utf-8"), ensure_ascii=False, indent=4)
save_data(src_texts, os.path.join(data_dir, "src_texts.json"))
save_data(trg_texts, os.path.join(data_dir, "trg_texts.json"))

# Load vocabulary
src_vocab = {word: i for i, word in enumerate(["<PAD>", "<UNK>", "<SOS>", "<EOS>"] + list(set([tok for text in src_texts for tok in tokenize_en(text)])))}
trg_vocab = {word: i for i, word in enumerate(["<PAD>", "<UNK>", "<SOS>", "<EOS>"] + list(set([tok for text in trg_texts for tok in tokenize_ko(text)])))}

# Save vocab
save_data(src_vocab, os.path.join(data_dir, "src_vocab.json"))
save_data(trg_vocab, os.path.join(data_dir, "trg_vocab.json"))

# Load dataset
src_texts = json.load(open(os.path.join(data_dir, "src_texts.json"), "r", encoding="utf-8"))
trg_texts = json.load(open(os.path.join(data_dir, "trg_texts.json"), "r", encoding="utf-8"))

# Custom Dataset Class
class TranslationDataset(Dataset):
    def __init__(self, src_texts, trg_texts, src_vocab, trg_vocab, max_len=50):
        self.src_texts = src_texts
        self.trg_texts = trg_texts
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab
        self.max_len = max_len
    
    def __len__(self):
        return len(self.src_texts)
    
    def __getitem__(self, idx):
        src_seq = self.text_to_tensor(self.src_texts[idx], self.src_vocab)
        trg_seq = self.text_to_tensor(self.trg_texts[idx], self.trg_vocab)
        return src_seq, trg_seq
    
    def text_to_tensor(self, text, vocab):
        tokens = tokenize_en(text) if vocab == src_vocab else tokenize_ko(text)
        token_ids = [vocab.get(token, vocab["<UNK>"]) for token in tokens]
        token_ids = token_ids[:self.max_len] + [vocab["<PAD>"]] * (self.max_len - len(token_ids))
        return torch.tensor(token_ids, dtype=torch.long)

# Create dataset and dataloader
dataset = TranslationDataset(src_texts, trg_texts, src_vocab, trg_vocab)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [6]:
# JSON 파일에서 로드한 원본 텍스트 확인
print("영어 문장 예시:")
print(src_texts[0])  # 첫 번째 영어 문장

print("\n한국어 문장 예시:")
print(trg_texts[0])  # 첫 번째 한국어 문장


영어 문장 예시:
Through the snow and sleet and hail, through the blizzard, through the gales, through the wind and through the rain, over mountain, over plain, through the blinding lightning flash, and the mighty thunder crash,

한국어 문장 예시:
폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘몰아쳐도


In [7]:
# 영어 문장의 토큰화 결과 확인
example_en = src_texts[0]
print("Tokenized English:")
print(tokenize_en(example_en))

# 한국어 문장의 토큰화 결과 확인
example_ko = trg_texts[0]
print("Tokenized Korean:")
print(tokenize_ko(example_ko))


Tokenized English:
['through', 'the', 'snow', 'and', 'sleet', 'and', 'hail', ',', 'through', 'the', 'blizzard', ',', 'through', 'the', 'gales', ',', 'through', 'the', 'wind', 'and', 'through', 'the', 'rain', ',', 'over', 'mountain', ',', 'over', 'plain', ',', 'through', 'the', 'blinding', 'lightning', 'flash', ',', 'and', 'the', 'mighty', 'thunder', 'crash', ',']
Tokenized Korean:
['폭설이', '내리고', '우박', ',', '진눈깨비가', '퍼부어도', '눈보라가', '몰아쳐도', '강풍이', '불고', '비바람이', '휘몰아쳐도']


In [8]:
word = "hello"
print(f"'{word}' 단어의 영어 vocab 인덱스: {src_vocab.get(word, src_vocab['<UNK>'])}")


'hello' 단어의 영어 vocab 인덱스: 80975


In [9]:
# 데이터셋의 첫 번째 샘플 확인
src_tensor, trg_tensor = dataset[0]

print("Source tensor (영어):")
print(src_tensor)  # 텐서 형태로 변환된 영어 문장 (길이 max_len)

print("\nTarget tensor (한국어):")
print(trg_tensor)  # 텐서 형태로 변환된 한국어 문장 (길이 max_len)


Source tensor (영어):
tensor([30083, 76073, 27677,  9609, 16718,  9609,  1628, 46741, 30083, 76073,
        80418, 46741, 30083, 76073, 79897, 46741, 30083, 76073, 74471,  9609,
        30083, 76073, 74891, 46741,  6220, 35610, 46741,  6220, 74953, 46741,
        30083, 76073, 14288, 24043, 65832, 46741,  9609, 76073, 37537, 71103,
         2302, 46741,     0,     0,     0,     0,     0,     0,     0,     0])

Target tensor (한국어):
tensor([175501, 443087, 646316, 126343, 522098, 426435, 262328, 249459, 576767,
        267776, 254033, 254510,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0])


In [10]:
# DataLoader에서 한 배치를 확인
for batch_idx, (src_batch, trg_batch) in enumerate(dataloader):
    print(f"Batch {batch_idx} - Source batch shape: {src_batch.shape}")
    print(f"Batch {batch_idx} - Target batch shape: {trg_batch.shape}")
    # 예시로 첫 배치만 확인하고 루프 탈출
    break


Batch 0 - Source batch shape: torch.Size([32, 50])
Batch 0 - Target batch shape: torch.Size([32, 50])


In [11]:
"aadf".text.lower

AttributeError: 'str' object has no attribute 'text'

['hello', 'world', '!', 'this', 'is', 'nlp', '.']